In [42]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve.meshes import MakeStructured2DMesh, MakeStructured3DMesh
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *
from netgen.geom2d import SplineGeometry
import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time
from scipy.optimize import minimize_scalar

In [43]:
def get_custom_frobenius_norm(A,theta):
   return A[0,0]**2 + A[1,1]**2 + (1/theta**2) * A[0,1]**2 + theta**2 * A[1,0]**2
    

In [44]:
## get problem parameters and geometry
problem = problems.problem4

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = "../"+problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 0.007
ord = 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional

rho = 1.23e-6
w,l = 23.5,90.0
theta = w/l

phi = lambda J: phi0/J

G = Parameter(G_target)

rect = SplineGeometry()
delta = 0.5
l_star = (1-delta)/2
pnts = [(delta/2,-1/2), (0.5,-1/2), (0.5,1/2), (delta/2,1/2)]

p1 = rect.AppendPoint(*pnts[0], hpref=1) # to close the rectangle
p2 = rect.AppendPoint(*pnts[1], hpref=1) # to close the rectangle
p3 = rect.AppendPoint(*pnts[2], hpref=1) # to close the rectangle
p4 = rect.AppendPoint(*pnts[3], hpref=1) # to close the rectangle



curves = [[["line",p1,p2],"bottom"],
        [["line",p2,p3],"right"],
        [["line",p3,p4],"top"],
        [["line",p4,p1],"left"]]


rect.Append(curves[0][0], bc = curves[0][1], leftdomain=1, rightdomain=0)
rect.Append(curves[1][0], bc = curves[1][1], leftdomain=1, rightdomain=0) #0.03
rect.Append(curves[2][0], bc = curves[2][1], leftdomain=1, rightdomain=0) #0.03
rect.Append(curves[3][0], bc = curves[3][1], leftdomain=1, rightdomain=0) #0.05



mesh = Mesh(rect.GenerateMesh(maxh=h)) #0.8
def Gel_energy_functional(u,beta):
    F = Id(2) + Grad(u)
    J = Det(F) * beta
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi)
    return 0.5*(G)* (get_custom_frobenius_norm(F,theta) + beta**2) + H * KBTV


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
fesu = VectorH1(mesh, order = ord, dirichlet = "left")
fesbeta = H1(mesh, order = ord)

fes = FESpace([fesu, fesbeta])
u, beta = fes.TrialFunction()
v, eta = fes.TestFunction()
BF = BilinearForm(fes)

# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.components[0].Set((0,0))
gfu.components[1].Set(1)

BF += Variation(Gel_energy_functional(u,beta).Compile()*dx)
Draw(mesh)
print(fes.ndof)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

71685


In [ ]:
max_iter_newton = 100
newton_damp = 0.3
softening_n = 100

tol_newton = 1e-8
gammas = np.flip(np.linspace(G_target, G_target*20 ,softening_n))
for num in range(len(gammas)):
    G.Set(gammas[num])
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()
    for iter in range(max_iter_newton):
        with TaskManager():
            BF.Apply(gfu.vec,res)
            BF.AssembleLinearization(gfu.vec)
            inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
            w.data = newton_damp * inv * res
        gfu.vec.data += -w
        print(InnerProduct(w,res), "Iter:",iter, "gamma:", gammas[num])
        if abs(InnerProduct(w,res)) < tol_newton:
            print("Tol achieved", InnerProduct(w,res))
            break
    print("Error newton:", InnerProduct(w,res), "Iter:",iter)   





0.0019050314918588735 Iter: 0 gamma: 2.6
0.0008719334588839904 Iter: 1 gamma: 2.6
0.00040817981833731143 Iter: 2 gamma: 2.6
0.00019390048247628426 Iter: 3 gamma: 2.6
9.301187685089622e-05 Iter: 4 gamma: 2.6
4.491171379694383e-05 Iter: 5 gamma: 2.6
2.1783955674328012e-05 Iter: 6 gamma: 2.6
1.0598892747499758e-05 Iter: 7 gamma: 2.6
5.167923773655359e-06 Iter: 8 gamma: 2.6
2.5235898977464465e-06 Iter: 9 gamma: 2.6
1.2335929719581835e-06 Iter: 10 gamma: 2.6
6.034468979921063e-07 Iter: 11 gamma: 2.6
2.9534218120325203e-07 Iter: 12 gamma: 2.6
1.445989288585166e-07 Iter: 13 gamma: 2.6
7.081279814975641e-08 Iter: 14 gamma: 2.6
3.468433106534509e-08 Iter: 15 gamma: 2.6
1.6990543713674118e-08 Iter: 16 gamma: 2.6
8.323728092214373e-09 Iter: 17 gamma: 2.6
Tol achieved 8.323728092214373e-09
Error newton: 8.323728092214373e-09 Iter: 17
6.081149357895223e-06 Iter: 0 gamma: 2.575050505050505
2.9909612008199276e-06 Iter: 1 gamma: 2.575050505050505
1.4694337805456474e-06 Iter: 2 gamma: 2.575050505050505

In [27]:
gfu2 = GridFunction(fesu)
gfu2.components[0].Set(gfu.components[0].components[1])
gfu2.components[1].Set(gfu.components[0].components[0])
Draw(gfu.components[0], mesh, "u", redraw=True)
Draw(gfu.components[1], mesh, "u", redraw=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [28]:
from ngsolve.meshes import MakeStructured3DMesh
d = 3
l = 90
w = 23.5
h = 1.0
mapping_unit_cube = lambda x,y,z: ((1-delta)*x/2+delta/2,y-0.5 , z)
mapping_phisical_gel = lambda x ,y ,z: (l*delta/2 + x*(1-delta)*l/2, -w/2 + y*w, z*d)
mesh_unit_cube = MakeStructured3DMesh(nx=int((1-delta)*l/2/h), ny=int(w/h), nz=int(d/h),mapping=mapping_unit_cube, hexes = False)
mesh_phisical_gel = MakeStructured3DMesh(
    nx=int((1-delta)*l/2/h), ny=int(w/h), nz=int(d/h),
    mapping=mapping_phisical_gel, hexes= False)

Draw(mesh_unit_cube)
Draw(mesh_phisical_gel)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [29]:
fesu_3d = VectorH1(mesh_unit_cube, order=ord)

gfu3d = GridFunction(fesu_3d)
gfu3d.components[0].Set(gfu.components[0].components[0])
gfu3d.components[1].Set(gfu.components[0].components[1])
gfu3d.components[2].Set(z*gfu.components[1])


In [30]:
Draw(gfu3d, mesh_unit_cube, "u")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [31]:
phisical_gel_fes = VectorH1(mesh_phisical_gel, order=ord)
phisical_gel_gfu = GridFunction(phisical_gel_fes)
phisical_gel_gfu.components[0].vec.data = l*gfu3d.components[0].vec
phisical_gel_gfu.components[1].vec.data = w*gfu3d.components[1].vec
phisical_gel_gfu.components[2].vec.data = d*gfu3d.components[2].vec
Draw(phisical_gel_gfu, mesh_phisical_gel, "u")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
en medio z* lambda_uni min equacion 18 ultimo paper

In [33]:
print(mesh_phisical_gel.maxh)

AttributeError: 'ngsolve.comp.Mesh' object has no attribute 'maxh'

Latex resumen del experimento